# Python version of radiative kernels
V. Cooper (2023)
- 0) Before running, make timeseries on std pressure levels
- 1) Compute climatology
- 2) Horizontal regridding
- 3) Feedback calcs converted from Pendergrass public scripts

Before running these scripts, postprocess the 3D fields into 
timeseries that are on pressure levels (not hybrid-sigma levels as 
in the native output). After timeseries files are prepared, they can
be loaded here to make a climatology, regrid onto kernel grid, then compute feedbacks.

Note: if changing the pressure levels from CMIP to something else,
- run "ncl scripts/calcp.ncl"
- run "ncl tools/calcdp_plev.ncl"
- run "ncl tools/t_kernel_to_plev.ncl"
- run "ncl tools/q_kernel_to_plev.ncl"

Note: the following 2 scripts are replaced by the timeseries creation, 
which already puts fields on pressure levels. No need to do these:
- don't run "ncl tools/convert_base_to_plevs.ncl"
- don't run "ncl tools/convert_change_to_plevs.ncl"

In [1]:
# import warnings
# warnings.filterwarnings('ignore')

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import xarray as xr
# import xesmf as xe
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import copy
import pandas as pd
import cmocean as cmo
from cartopy.util import add_cyclic_point
import seaborn as sns
%matplotlib inline

import xesmf as xe

# Compute climatology

In [2]:
## these are the dummy files for format and grid matching
path = './'

## these are demodata, will be replaced later after we match
## the file formatting
basefields = xr.open_dataset(path +'demodata/'+ 'basefields.nc~')
changefields = xr.open_dataset(path +'demodata/'+ 'changefields.nc~')
PSdemo = xr.open_dataset(path + 'kernels/PS.nc')
pdiff = xr.open_dataset(path + 'dp_plev.nc')
gw = xr.open_dataset(path +'kernels/'+ 't.kernel.nc').gw

varlist = ['FSDS','FSNS','Q','T','FLNS',
           'FLNSC','FLNT','FLNTC','FSNSC',
           'FSNT','FSNTC','PS'] #'TS' will be loaded by default

### Select cases here

In [3]:
path = '/glade/scratch/vcooper/processed/'

## SELECT BASECASE
basecase = 'f.e1221.F_2000_CAM5.f19_f19.LGMRholo.002' ## CAM5

# basecase = 'f.e1221.F_2000.f19_f19.LGMRholo.002' ## CAM4
#basecase = 'f.e1221.F_2000.f19_f19.LGMRholo.003' ## CAM4 v3

## SELECT CHANGECASE
# case = 'f.e1221.F_2000_CAM5.f19_f19.longrunmip-0p5.002'
# case = 'f.e1221.F_2000_CAM5.f19_f19.longrunmip.002'
# case = 'f.e1221.F_2000_CAM5.f19_f19.LGMR-0p5.002'
# case = 'f.e1221.F_2000_CAM5.f19_f19.LGMRlgm.001'
# case = 'f.e1221.F_2000_CAM5.f19_f19.lgmDA.002'
case = 'f.e1221.F_2000_CAM5.f19_f19.lgmDA-0p5.002'

# case = 'f.e1221.F_2000.f19_f19.longrunmip-0p5.002'
# case = 'f.e1221.F_2000.f19_f19.longrunmip2x.002'
# case = 'f.e1221.F_2000.f19_f19.LGMR-0p5.002'
# case = 'f.e1221.F_2000.f19_f19.LGMRlgm.001'

changecase = case

## load in the full timeseries
fname = basecase + '.cam.h0.tseries_interp.nc'
basecam_out = xr.open_dataset(path + basecase +'/'+fname)
fname = case + '.cam.h0.tseries_interp.nc'
cam_out = xr.open_dataset(path + case +'/'+fname)

In [4]:
## this is stupid code to fix the months
init_bound = np.array(xr.cftime_range(start="0006", periods=1, freq="MS", calendar="noleap")[0])
dumtime = np.hstack([init_bound,cam_out.time.values])
newtime = dumtime[0:-1]
cam_out['time'] = newtime
basecam_out['time'] = newtime
cam_climo = cam_out.groupby('time.month').mean()
basecam_climo = basecam_out.groupby('time.month').mean()

In [5]:
## OLD VERSION: save these files (not necessary anymore)
# path = '/glade/scratch/vcooper/processed/'
# savename = case + '.cam.h0.climo.nc'
# cam_climo.to_netcdf(path + case + '/' + savename)
# print('finished save of ', savename)

In [6]:
### Change some fields to match demodata
basecase = basecam_climo.rename(
    {'month':'time','T':'temp','TS':'ts','lev':'lev_p'})

changecase = cam_climo.rename(
    {'month':'time','T':'temp','TS':'ts','lev':'lev_p'})

In [7]:
diff = changecase - basecase

# Regrid horizontal fields to match kernel 1 degree resolution

In [8]:
## xesmf issue with date and datesec, avoiding it here:
regridvars = ['temp',
                         # 'date',
                         # 'datesec',
                         'Q',
                         'PS',
                         'ts',
                         'FSDS',
                         'FSNS',
                         'FLNS',
                         'FLNSC',
                         'FLNT',
                         'FLNTC',
                         'FSNSC',
                         'FSNT',
                         'FSNTC']

In [9]:
newgrid = basefields.ts[0].to_dataset()
newgrid = basefields

data_for_regridding = basecase.ts[0].to_dataset()
data_for_regridding = basecase

regridder = xe.Regridder(data_for_regridding, newgrid,
                         method='bilinear',
                         periodic=True,
                         # extrap_method='inverse_dist',extrap_num_src_pnts=8,
                         filename='bilinear_2deg_to_1deg.nc',
                         reuse_weights=True)

base_regrid = regridder(basecase[regridvars])
diff_regrid = regridder(diff[regridvars])

## avoiding xesmf issue
base_regrid['date'] = basecase['date']
base_regrid['datesec'] = basecase['datesec']

diff_regrid['date'] = diff['date']
diff_regrid['datesec'] = diff['datesec']

Reuse existing file: bilinear_2deg_to_1deg.nc
using dimensions ('lat', 'lon') from data variable temp as the horizontal dimensions for this dataset.


/glade/u/home/vcooper/miniconda3/envs/plato/lib/python3.8/site-packages/xesmf/frontend.py:477: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


using dimensions ('lat', 'lon') from data variable temp as the horizontal dimensions for this dataset.


In [10]:
pdiff

<xarray.Dataset>
Dimensions:  (time: 12, plev: 17, lat: 192, lon: 288)
Coordinates:
  * time     (time) object 2006-12-16 10:30:00 ... 2006-12-16 10:30:00
  * plev     (plev) int32 100000 92500 85000 70000 60000 ... 5000 3000 2000 1000
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Data variables:
    dp       (time, plev, lat, lon) float64 ...
Attributes:
    creation_date:  Tue Apr  4 19:16:44 MDT 2023
    source_file:    kernels/PS.nc
    title:          Change in pressure on pressure coordinate

In [11]:
## add some variables to file that seem to not change
## note: pdiffs are constant because they are diff between 
## the native kernels and the CMIP5 standard levels
base_regrid['plev'] = base_regrid.lev_p * 100
base_regrid['gw'] = ('lat', gw.values)
base_regrid['pdiff'] = (['time','lev_p','lat','lon'], pdiff.dp.values)

## note that pdiff is the same for the changefields
diff_regrid['plev'] = diff_regrid.lev_p * 100
diff_regrid['gw'] = ('lat', gw.values)
diff_regrid['pdiff'] = (['time','lev_p','lat','lon'], pdiff.dp.values)

In [12]:
## OLD VERSION: Save these files
# ## save data into correct directory
# # finaldir = '/glade/work/vcooper/kernels/cam5-kernels/'
# finaldir = '/glade/work/vcooper/kernels/2023_cam5/cam5-kernels/'
# fname = 'basefields.plev.nc'
# base_regrid.to_netcdf(finaldir + fname)
# fname = 'changefields.plev.nc'
# diff_regrid.to_netcdf(finaldir + fname)

# ## save here too since 2D variables use this file
# # finaldir = '/glade/work/vcooper/kernels/cam5-kernels/demodata/'
# finaldir = '/glade/work/vcooper/kernels/2023_cam5/cam5-kernels/demodata/'
# fname = 'basefields.nc'
# base_regrid.to_netcdf(finaldir + fname)
# fname = 'changefields.nc'
# diff_regrid.to_netcdf(finaldir + fname)
# print('done')

# Calculate all feedbacks
- This portion is adapted from Pendergrass scripts

In [13]:
# File with the changes in climate: (ts, temp) (TS,T,Q)
varlist2d = ['ts','gw','FSNS', 'FSDS', 'FSNT']

# changefile=xr.open_dataset('./demodata/changefields.nc')[varlist2d]
# changefile3d=xr.open_dataset('./changefields.plev.nc')
# basefile=xr.open_dataset('./demodata/basefields.nc')[varlist2d]
# basefile3d=xr.open_dataset('./basefields.plev.nc')

changefile=diff_regrid#[varlist2d]
changefile3d=diff_regrid
basefile=base_regrid#[varlist2d]
basefile3d=base_regrid


###################

## Read air temperature kernel 
# ta_kernel_hybrid=ncread('kernels/t.kernel.nc','FLNT');
ta_kernel=xr.open_dataset('t.kernel.plev.nc')

## VTC add section to read in pressure levels
p_Pa=ta_kernel.plev
p_hPa=ta_kernel.lev_p 

## this must be generated by calcdp_plev.ncl script
pdiff=xr.open_dataset('dp_plev.nc').dp/100

# p=repmat(permute(repmat(p_hPa,[1 12]),[3 4 1 2]),[288 192 1 1]);
p = p_hPa.values[np.newaxis, :, np.newaxis, np.newaxis] * np.ones(ta_kernel.FLNT.shape)

In [14]:
###################

## Read in coordinate info
lat=xr.open_dataset('./kernels/PS.nc').lat
lon=xr.open_dataset('./kernels/PS.nc').lon
gw=xr.open_dataset('./kernels/t.kernel.nc').gw ## Gaussian weights for CESMgrid
# lev=ncread('kernels/t.kernel.nc','lev'); ## dont need

## Make an area weighting matrix
weight=np.tile(gw.values[:,np.newaxis], len(lon))
weight=weight/np.nansum(weight)
# print(weight.sum())

## Read surface temperature change
dts=changefile.ts

## Calculate the change in global mean surface temperature
dts_globalmean= (dts * weight).sum(dim=('lat','lon')).mean(dim='time')
print('Global mean dTS: ', dts_globalmean.values)
                 

Global mean dTS:  -4.224239359571119


In [15]:
## Temperature feedback calculation

## Read TOA Longwave surface temperature kernel
ts_kernel=xr.open_dataset('./kernels/ts.kernel.nc').FLNT

## Multiply monthly mean TS change by the TS kernels (function of
## lat, lon, month) (units W/m2)
dLW_ts=ts_kernel *dts
#dLW_ts.mean(dim='time').plot()

## Read air temperature change [lon,lat,level,month]
dta=changefile3d.temp


In [16]:
## Non-pressure level version:
## Read midpoint pressure for each grid cell (lat,lon,level,month), [Pa]
## VTC adjusted this above to be pressure levels
## p=ncread('p_sigma.nc','pmid')/100; %[hPa] 

In [17]:
## Crude tropopause estimate: 100 hPa in the tropics, lowering with
## cosine to 300 hPa at the poles.
x=np.cos(np.deg2rad(lat))
p_tropopause_zonalmean=300-200*x
## VTC
##p_tropopause= ...
##    repmat(permute(repmat(permute(repmat(p_tropopause_zonalmean', ...
##                                         [length(lon) 1]),[1 2 3]),[1 ...
##                    1 length(lev)]),[1 2 3 4]),[1 1 1 12]);
p_tropopause_zonalmean
p_tropopause = p_tropopause_zonalmean.values[
    np.newaxis, np.newaxis,:, np.newaxis] * np.ones(ta_kernel.FLNT.shape)
p_tropopause = xr.DataArray(p_tropopause,dims=changefile3d.dims, coords = changefile3d.coords)

In [18]:
## Set the temperature change to zero in the stratosphere (mask out stratosphere)
dta=xr.where(p>=p_tropopause, dta, np.nan)

## Convolve air temperature kernel with air temperature change
## VTC
## dLW_ta=squeeze(sum(ta_kernel.*dta,3));
dLW_ta=ta_kernel.FLNT * dta.values * pdiff.values

In [19]:
## Add the surface and air temperature response; Take the annual
## average and global area average 
dLW_t_globalmean = -(
    (dLW_ta.sum(dim='plev') + dLW_ts).mean(dim='time') * weight).sum()

## Divide by the global annual mean surface warming (units: W/m2/K)
t_feedback=dLW_t_globalmean / dts_globalmean

print('Temperature feedback: ', str(t_feedback.values), ' W m^-2 K^-1')

Temperature feedback:  -3.1530739651180153  W m^-2 K^-1


# Albedo Feedback

In [20]:
## Collect surface shortwave radiation fields to calculate albedo.
## Alternatively, you might already have the change in albedo - that
## would work too.
SW_sfc_net_1=basefile.FSNS
SW_sfc_down_1=basefile.FSDS
SW_sfc_net_2=changefile.FSNS+SW_sfc_net_1
SW_sfc_down_2=changefile.FSDS+SW_sfc_down_1

alb1= 1 - SW_sfc_net_1/SW_sfc_down_1
alb1 = xr.where(np.isnan(alb1), 0, alb1)

alb2= 1 - SW_sfc_net_2/SW_sfc_down_2
alb2 = xr.where(np.isnan(alb2), 0, alb2)

dalb=(alb2-alb1)*100

alb_kernel = xr.open_dataset('./kernels/alb.kernel.nc').FSNT
dSW_alb = alb_kernel * dalb


In [21]:
## average and global area average 
dSW_alb_globalmean = (dSW_alb.mean(dim='time') * weight).sum()
alb_feedback = dSW_alb_globalmean / dts_globalmean

print('Surf. albedo feedback: ', str(alb_feedback.values), ' W m^-2 K^-1')

Surf. albedo feedback:  0.5166022198586437  W m^-2 K^-1


# Water vapor feedback

In [22]:
## Calculate the change in moisture per degree warming at constant relative humidity. 
q1=basefile3d.Q
t1=basefile3d.temp

In [23]:
## addpath scripts/
## VTC script for
def calcsatspechum(t_in, p_in):
    ## T is temperature, P is pressure in hPa 

    ## Formulae from Buck (1981):
    es = (1.0007+(3.46e-6 * p_in)) * 6.1121 * (
        np.exp(17.502*(t_in - 273.15) / (240.97+(t_in - 273.15))))
    
    wsl = 0.622 * es / (p_in - es) ## saturation mixing ratio wrt liquid water (g/kg)
    
    es = (1.0003 + ( 4.18e-6 * p_in)) * 6.1115 * (
        np.exp(22.452 * (t_in - 273.15) / (272.55 + (t_in - 273.15))))
    
    wsi = 0.622 * es / (p - es) ## saturation mixing ratio wrt ice (g/kg)
    
    ws = wsl
    
    ws = xr.where(t_in < 273.15, wsi, ws)
    
    qs = ws / (1+ws) ## saturation specific humidity, g/kg
    return(qs)


In [24]:
qs1 = calcsatspechum(t1,p) #g/kg
qs2 = calcsatspechum(t1+dta,p) #g/kg
dqsdt = (qs2 - qs1) / dta
rh = q1 / qs1
dqdt = rh * dqsdt

## Read kernels
q_LW_kernel = xr.open_dataset('q.kernel.plev.nc').FLNT
q_SW_kernel = xr.open_dataset('q.kernel.plev.nc').FSNT

## Normalize kernels by the change in moisture for 1 K warming at
## constant RH
q_LW_kernel = q_LW_kernel.values / dqdt;
q_SW_kernel = q_SW_kernel.values / dqdt;

## Read the change in moisture
dq = changefile3d.Q

In [25]:
## Set the moisture change to zero in the stratosphere (mask out stratosphere)
dq = xr.where(p>=p_tropopause, dq, np.nan)

In [26]:
## Convolve moisture kernel with change in moisture
dLW_q = q_LW_kernel * dq.values * pdiff.values
dSW_q = q_SW_kernel * dq.values * pdiff.values

In [27]:
## Add the LW and SW responses. Note the sign convention difference
## between LW and SW!
dR_q_globalmean = (
    (-dLW_q + dSW_q).sum(dim='lev_p').mean(dim='time') * weight).sum()

## Divide by the global annual mean surface warming (units: W/m2/K)
q_feedback = dR_q_globalmean / dts_globalmean

print('Water vapor feedback: ', str(q_feedback.values), ' W m^-2 K^-1')



Water vapor feedback:  1.3140729639589164  W m^-2 K^-1


# WV with log of Q

In [28]:
## Calculate the change in moisture per degree warming at constant relative humidity. 
## Run the accompanying NCL script with your input files, or
## implement here.                                                             

In [29]:
q0 = q1 ## AP used q1 for this above #basefile3d.Q #kg/kg 

## all of these are set above
#t1 = basefile3d.temp set above
#dta = changefile3d.temp ## set above
#qs1 = calcsatspechum(t1,p); % g/kg
#qs2 = calcsatspechum(t1+dta,p); % g/kg
#dqsdt = (qs2 - qs1)./dta;

## slightly different from above
rh = 1000*q0 / qs1
dqdt = rh * dqsdt ## assume constant RH
dlogqdt= dqdt / (1000 * q0) ## convert denominator to g/kg

## Re-read kernels, 
## normalize by the change in moisture for 1 K warming at
## constant RH using log Q
q_LW_kernel = xr.open_dataset('q.kernel.plev.nc').FLNT.values
logq_LW_kernel = q_LW_kernel / dlogqdt
q_SW_kernel = xr.open_dataset('q.kernel.plev.nc').FSNT.values
logq_SW_kernel = q_SW_kernel / dlogqdt

In [30]:
## all set above
## Read the change in moisture
## dq=ncread(changefile3d,'Q');
## Mask out the stratosphere
##dq=dq.*(p>=p_tropopause);

dlogq = dq / q0


In [31]:
## Convolve moisture kernel with change in moisture
dLW_logq = logq_LW_kernel.values * dlogq * pdiff.values
dSW_logq = logq_SW_kernel.values * dlogq * pdiff.values

## Add the LW and SW responses. 
## Note the sign convention difference LW and SW
dR_logq_globalmean = (
    (-dLW_logq + dSW_logq).sum(dim='lev_p').mean(dim='time') * weight).sum()

## Divide by the global annual mean surface warming (units: W/m2/K)
logq_feedback = dR_logq_globalmean / dts_globalmean

print('logQ WV feedback: ', str(logq_feedback.values), ' W m^-2 K^-1')
print('linQ WV feedback: ', str(q_feedback.values), ' W m^-2 K^-1')

logQ WV feedback:  1.3140729639589164  W m^-2 K^-1
linQ WV feedback:  1.3140729639589164  W m^-2 K^-1


# Planck Feedback

In [32]:
## Project surface temperature change into height 
##VTC
##dts3d=repmat(permute(dts,[1 2 4 3]),[1 1 30 1]);
dts3d = dts + changefile3d.temp-changefile3d.temp

## Mask stratosphere
dt_planck = xr.where(p>=p_tropopause, dts3d, np.nan)


In [33]:
## Convolve air temperature kernel with 3-d surface air temp change
##VTC
##dLW_planck=squeeze(sum(ta_kernel_hybrid.*dt_planck,3));
# dLW_planck = squeeze(sum(ta_kernel * dt_planck.*pdiff,3))
dLW_planck=ta_kernel.FLNT * dt_planck.values * pdiff.values

## Take the annual average and global area average; incorporate the
## part due to surface temperature change itself 
dLW_planck_globalmean = -(
    (dLW_planck.sum(dim='plev') + dLW_ts).mean(dim='time') * weight).sum()

## Divide by the global annual mean surface warming (units: W/m2/K)
planck_feedback=dLW_planck_globalmean / dts_globalmean


In [34]:
## Lapse rate feedback                                                                                                                                                                 
## Calculate the departure of temperature change from the surface
## temperature change
dt_lapserate=xr.where(p>=p_tropopause, dta-dt_planck, np.nan)

## Convolve air temperature kernel with 3-d surface air temp change
## VTC
## dLW_lapserate=squeeze(sum(ta_kernel.*dt_lapserate,3));
# dLW_lapserate=squeeze(sum(ta_kernel.*dt_lapserate.*pdiff,3));
dLW_lapserate = ta_kernel.FLNT * dt_lapserate.values * pdiff.values

## Take the annual average and global area average 
dLW_lapserate_globalmean = -(
    (dLW_lapserate.sum(dim='plev')).mean(dim='time') * weight).sum()

## Divide by the global annual mean surface warming (units: W/m2/K)
lapserate_feedback = dLW_lapserate_globalmean / dts_globalmean

print('Planck feedback: ', 
      str(planck_feedback.values), ' W m^-2 K^-1')
print('Lapse rate feedback: ', 
      str(lapserate_feedback.values), ' W m^-2 K^-1')

### SANITY CHECK: Do the Planck and lapse-rate feedbacks add up to
### the total temperature feedback? (They should)

## Planck + lapse rate feedback
total_t_feedback = planck_feedback+lapserate_feedback

print('Temperature feedback: ',
      str(t_feedback.values), ' W m^-2 K^-1')
print('Planck+lapse rate components: ',
      str(total_t_feedback.values), ' W m^-2 K^-1')


Planck feedback:  -3.2079333515340798  W m^-2 K^-1
Lapse rate feedback:  0.054859386416064984  W m^-2 K^-1
Temperature feedback:  -3.1530739651180153  W m^-2 K^-1
Planck+lapse rate components:  -3.153073965118015  W m^-2 K^-1


## Cloud Feedback

In [35]:
## STEP 1. Calculate total-sky and clear-sky feedbacks
lev = ta_kernel.plev.values ## this is in Pa

In [36]:
## Read TOA Longwave surface temperature kernel
ts_kernel_clearsky = xr.open_dataset('./kernels/ts.kernel.nc').FLNTC

## Multiply monthly mean TS change by the TS kernels 
## (function of lat, lon, month) (units W/m2)
dLW_ts_cs = ts_kernel_clearsky * dts 

## Read clear-sky air temperature kernel
ta_kernel_clearsky = xr.open_dataset('./t.kernel.plev.nc').FLNTC

## Convolve air temperature kernel with air temperature change
dLW_ta = dLW_ta.sum(dim='plev')
dLW_ta_cs = (ta_kernel_clearsky.values * dta * pdiff.values).sum(dim='lev_p')

In [37]:
## ALBEDO clear sky

## Read TOA albedo kernel
alb_kernel_clearsky = xr.open_dataset('./kernels/alb.kernel.nc').FSNTC

dSW_alb_cs = alb_kernel_clearsky.values * dalb

In [38]:
## WATER VAPOR clear sky

## read kernels
q_LW_kernel = xr.open_dataset('./q.kernel.plev.nc').FLNT
q_SW_kernel = xr.open_dataset('./q.kernel.plev.nc').FSNT
q_LW_kernel_clearsky = xr.open_dataset('./q.kernel.plev.nc').FLNTC
q_SW_kernel_clearsky = xr.open_dataset('./q.kernel.plev.nc').FSNTC

## Normalize kernels by the change in moisture for 1 K warming at
## constant RH (linear)
rh = q1 / qs1
dqdt = rh * dqsdt ## from above, reset to linear method
q_LW_kernel = q_LW_kernel.values / dqdt
q_SW_kernel = q_SW_kernel.values / dqdt
q_LW_kernel_clearsky = q_LW_kernel_clearsky.values / dqdt
q_SW_kernel_clearsky = q_SW_kernel_clearsky.values / dqdt

## Convolve moisture kernel with change in moisture
dLW_q = (q_LW_kernel.values * dq * pdiff.values).sum(dim='lev_p')
dSW_q = (q_SW_kernel.values * dq * pdiff.values).sum(dim='lev_p')
dLW_q_cs = (q_LW_kernel_clearsky.values * dq * pdiff.values).sum(dim='lev_p')
dSW_q_cs = (q_SW_kernel_clearsky.values * dq * pdiff.values).sum(dim='lev_p')

In [39]:
### Change in Cloud Radiative Effect (CRE) 
d_sw = changefile3d.FSNT
d_sw_cs = changefile3d.FSNTC
d_lw = changefile3d.FLNT
d_lw_cs = changefile3d.FLNTC

d_cre_sw = d_sw_cs - d_sw
d_cre_lw = d_lw_cs - d_lw


In [40]:
## THIS WOULD NEED TO BE ADJUSTED
### Cloud masking of radiative forcing
ghgfile = './forcing/ghg.forcing.nc'
sw = xr.open_dataset(ghgfile).FSNT
sw_cs = xr.open_dataset(ghgfile).FSNTC
lw = xr.open_dataset(ghgfile).FLNT
lw_cs = xr.open_dataset(ghgfile).FLNTC
ghg_sw = sw_cs-sw
ghg_lw = lw_cs-lw

aerosolfile = './forcing/aerosol.forcing.nc';
sw = xr.open_dataset(aerosolfile).FSNT
sw_cs = xr.open_dataset(aerosolfile).FSNTC
lw = xr.open_dataset(aerosolfile).FLNT
lw_cs = xr.open_dataset(aerosolfile).FLNTC
aerosol_sw = sw_cs - sw
aerosol_lw = lw_cs - lw

## MAKE FORCING ZERO because there is no forcing in these experiments
cloud_masking_of_forcing_sw = 0 # aerosol_sw + ghg_sw
cloud_masking_of_forcing_lw = 0 # aerosol_lw + ghg_lw


In [41]:
### Cloud feedback
### CRE + cloud masking of radiative forcing + corrections for each feedback

dLW_cloud = -d_cre_lw + cloud_masking_of_forcing_lw + (
    dLW_q_cs - dLW_q.values) + (dLW_ta_cs - dLW_ta.values) + (dLW_ts_cs - dLW_ts)
dSW_cloud = -d_cre_sw + cloud_masking_of_forcing_sw + (
    dSW_q_cs - dSW_q.values) + (dSW_alb_cs-dSW_alb)


In [42]:
# ## LW CLOUD
# height=[ 
#             (-d_cre_lw.mean(dim='time') * weight).sum(),
#             (cloud_masking_of_forcing_lw.mean(dim='time') * weight).sum(),
#             ((dLW_q_cs - dLW_q.values).mean(dim='time') * weight).sum(),
#             ((dLW_ta_cs - dLW_ta.values).mean(dim='time') * weight).sum(),
#             ((dLW_ts_cs - dLW_ts).mean(dim='time') * weight).sum(),
#             (dLW_cloud.mean(dim='time') * weight).sum(),
#             ]
# plt.bar(x=np.arange(len(height)),
#         height=height)
# plt.show()


# ## SW Cloud
# height=[ 
#             (-d_cre_sw.mean(dim='time') * weight).sum(),
#             (cloud_masking_of_forcing_sw.mean(dim='time') * weight).sum(),
#             ((dSW_q_cs - dSW_q.values).mean(dim='time') * weight).sum(),
#             ((dSW_alb_cs - dSW_alb.values).mean(dim='time') * weight).sum(),
#             (dSW_cloud.mean(dim='time') * weight).sum(),
#             ]
# plt.bar(x=np.arange(len(height)),
#         height=height)

In [43]:
## Take global and annual averages
dLW_cloud_globalmean = (-dLW_cloud.mean(dim='time') * weight).sum()
dSW_cloud_globalmean = ( dSW_cloud.mean(dim='time') * weight).sum()

## Divide by global, annual mean temperature change to get W/m2/K
lw_cloud_feedback = dLW_cloud_globalmean / dts_globalmean
sw_cloud_feedback = dSW_cloud_globalmean / dts_globalmean

print('LW Cloud feedback: ',
      str(lw_cloud_feedback.values), ' W m^-2 K^-1')
print('SW Cloud feedback: ',
      str(sw_cloud_feedback.values), ' W m^-2 K^-1')

LW Cloud feedback:  0.2194385924650693  W m^-2 K^-1
SW Cloud feedback:  0.5139111261261101  W m^-2 K^-1


# All Feedbacks

In [44]:
print('Planck feedback: ', 
      str(planck_feedback.values), ' W m^-2 K^-1')
print('Lapse rate feedback: ', 
      str(lapserate_feedback.values), ' W m^-2 K^-1')
print('Water vapor feedback: ', 
      str(q_feedback.values), ' W m^-2 K^-1')
print('Surf. albedo feedback: ', 
      str(alb_feedback.values), ' W m^-2 K^-1')
print('LW Cloud feedback: ',
      str(lw_cloud_feedback.values), ' W m^-2 K^-1')
print('SW Cloud feedback: ',
      str(sw_cloud_feedback.values), ' W m^-2 K^-1')

sumall = planck_feedback + lapserate_feedback + q_feedback + alb_feedback + (
    lw_cloud_feedback + sw_cloud_feedback)
print('\n\nSum all feedbacks: ',
      str(sumall.values), ' W m^-2 K^-1')

## extras
print('\n\nTemperature feedback: ',
      str(t_feedback.values), ' W m^-2 K^-1')
print('Planck+lapse rate components: ',
      str(total_t_feedback.values), ' W m^-2 K^-1')

print('\n\nlogQ WV feedback: ', str(logq_feedback.values), ' W m^-2 K^-1')
print('linQ WV feedback: ', str(q_feedback.values), ' W m^-2 K^-1')

Planck feedback:  -3.2079333515340798  W m^-2 K^-1
Lapse rate feedback:  0.054859386416064984  W m^-2 K^-1
Water vapor feedback:  1.3140729639589164  W m^-2 K^-1
Surf. albedo feedback:  0.5166022198586437  W m^-2 K^-1
LW Cloud feedback:  0.2194385924650693  W m^-2 K^-1
SW Cloud feedback:  0.5139111261261101  W m^-2 K^-1


Sum all feedbacks:  -0.5890490627092753  W m^-2 K^-1


Temperature feedback:  -3.1530739651180153  W m^-2 K^-1
Planck+lapse rate components:  -3.153073965118015  W m^-2 K^-1


logQ WV feedback:  1.3140729639589164  W m^-2 K^-1
linQ WV feedback:  1.3140729639589164  W m^-2 K^-1


In [45]:
np.array([planck_feedback,
          lapserate_feedback,
          q_feedback,
          alb_feedback,
          lw_cloud_feedback,
          sw_cloud_feedback])

array([-3.20793335,  0.05485939,  1.31407296,  0.51660222,  0.21943859,
        0.51391113])

In [46]:
!pwd -P

/glade/work/vcooper/kernels/2023_cam5/cam5-kernels
